# Construction of a Linear program for a given degree
Equation to Solve
$$\ \vert x \vert_1^r \cdot f_0(x) = \sum_i^n f_i(x)g_i(x)$$
we use the following packages
- Dynamic Polynomails: for multivariate Polynomials
- LinearAlgebra: for Linear Algebra
- JuMP: Linear Programming
- HiGHS: a solver for linear programming

In [1]:
using DynamicPolynomials,LinearAlgebra
using JuMP
import HiGHS

## Step by Step Instruction
As Input we need the following:
- $r\in \mathbb{N}$
- An array of homogenous polynomials $F$ 

At first we create a Example to explain its steps via this example

In [2]:
@polyvar x[1:3] ## gives us 3 Variables for polynomial construction

(PolyVar{true}[x₁, x₂, x₃],)

Now we can create a bunch of polynomials in $\mathbb R[x_1,x_2,x_3]$

In [3]:
f₀ = 3x[1] - 2x[2] - 2x[3]
f₁ = 1 + 0*x[1]
f₂ = x[1]-x[2]
f₃ = x[1]-x[3]
f₄ = x[1]^2 - 4x[2]x[3]

x₁² - 4x₂x₃

Which means we can create our Inputs 

In [4]:
r=1
F=[f₀,f₁,f₂,f₃,f₄]

5-element Vector{Polynomial{true, Int64}}:
 3x₁ - 2x₂ - 2x₃
 1
 x₁ - x₂
 x₁ - x₃
 x₁² - 4x₂x₃

We first calcualte the coefficients $l_\alpha$ on the Left site of the equation 

In [5]:
base = monomials(x,1) ## Every possible monomial of degree 1

3-element MonomialVector{true}:
 x₁
 x₂
 x₃

In [6]:
polynomial(base) ## This gives us a easy way to calculate the left site

x₁ + x₂ + x₃

In [7]:
left = polynomial(base)^r*f₀ ##The left site of the Equation

3x₁² + x₁x₂ + x₁x₃ - 2x₂² - 4x₂x₃ - 2x₃²

Now we need to only extract the coefficients

In [8]:
l = coefficients(left,monomials(x,2)) ##monomials(x,2) calculates the monomials base for  

6-element Vector{Int64}:
  3
  1
  1
 -2
 -4
 -2

## Right Side
Now we know that $f_i\cdot g_i=\sum_j m_j \cdot f_i$. Where $m_j$ are monomials conained in $g_i$

In [9]:
d₀ = maxdegree(f₀)
d₁ = maxdegree(f₁)
m = monomials(x,d₀+r-d₁) 

6-element MonomialVector{true}:
 x₁²
 x₁x₂
 x₁x₃
 x₂²
 x₂x₃
 x₃²

So now we want to calculate the coefficients from $m_j \cdot g_i$.

In [10]:
m*f₁

6-element Vector{Polynomial{true, Int64}}:
 x₁²
 x₁x₂
 x₁x₃
 x₂²
 x₂x₃
 x₃²

The monomomial base of this have degree $d_0+r$. Which means we can calculate ist coefficients.

In [11]:
C₁=[]
for mᵢ in m*f₁
    push!(C₁,coefficients(mᵢ,monomials(x,d₀+r)))
end
C₁

6-element Vector{Any}:
 [1, 0, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0]
 [0, 0, 1, 0, 0, 0]
 [0, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, 1, 0]
 [0, 0, 0, 0, 0, 1]

This now has to be convertert into a Matrix, as opposed to a vector of vectors.

In [12]:
C₁=transpose(reduce(vcat,transpose.(C₁))) ## This Transposed every Vector and then adds then together. Before Transposing evrything again

6×6 transpose(::Matrix{Int64}) with eltype Int64:
 1  0  0  0  0  0
 0  1  0  0  0  0
 0  0  1  0  0  0
 0  0  0  1  0  0
 0  0  0  0  1  0
 0  0  0  0  0  1

Now to find coefficients for $$\vert x \vert_1^r \cdot f_0(x) = g_1 \cdot f_1$$ 
we only have to find x so that $$ C_1 x = l$$ which would be our coefficients of $g_1$.

## Multiple Polynomials

To generalise this to multiple Polynomials we only have to interate over them.

In [13]:
M=[]
for fᵢ in F[2:5]
    dᵢ = maxdegree(fᵢ)
    m = monomials(x,d₀+r-dᵢ) 
    Cᵢ=[]
    for mᵢ in m*fᵢ
        push!(Cᵢ,coefficients(mᵢ,monomials(x,d₀+r)))
    end
    Cᵢ=transpose(reduce(vcat,transpose.(Cᵢ)))
    push!(M,Cᵢ)
end
M=transpose(reduce(vcat,transpose.(M)))

6×13 transpose(::Matrix{Int64}) with eltype Int64:
 1  0  0  0  0  0   1   0   0   1   0   0   1
 0  1  0  0  0  0  -1   1   0   0   1   0   0
 0  0  1  0  0  0   0   0   1  -1   0   1   0
 0  0  0  1  0  0   0  -1   0   0   0   0   0
 0  0  0  0  1  0   0   0  -1   0  -1   0  -4
 0  0  0  0  0  1   0   0   0   0   0  -1   0

Solving $$ Mx=l$$ gives us coefficients of $g$

# Solving a Linear Programm
At first we have to define the LP to solve.
## Creating the LP
We first need to calculate the Size of the Matrix

In [14]:
n=size(M,2) ## M is a 6 x 13 Matrix
m=size(M,1)

6

In [27]:
LP=Model(HiGHS.Optimizer) ## initializing the Linear Programm
@variable(LP, u[1:n]>=0) ## Defining a n-dimensional Variable an automaticly constraint it to be nonnegative
print(LP)

In [28]:
@constraint(LP,M*u.==l) ## Define constraint  .== means that it is componentwise equal
print(LP)

Now we only have to find a solution to this LP
## Solving a Linear Program
this one is easy

In [29]:
optimize!(LP)

Presolving model
4 rows, 9 cols, 14 nonzeros
2 rows, 7 cols, 10 nonzeros
2 rows, 4 cols, 6 nonzeros
2 rows, 4 cols, 6 nonzeros
Presolve : Reductions: rows 2(-4); columns 4(-9); elements 6(-14)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(5) 0s
          1     0.0000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.02


The Solver tells us it has found a optimal solution which we can now extract

In [31]:
Solution = JuMP.value.(u)

13-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 2.0
 0.0
 1.0
 0.0
 2.0
 1.0

Now that we got the coefficients vector of our solution we only have to calculate our solution polynomials

In [16]:
function getLP(poly,arrayOfPolynomials,var)
    d = maxdegree(poly)
    base=monomials(var,d)
    b=coefficients(poly,base)
    c=[]
    nVars=[]
    for f in arrayOfPolynomials
        deg=d - maxdegree(f)
        push!(nVars,size(monomials(var,deg))[1])
        l = []
        for g in monomials(var,deg)
            basepoly=g*f
            push!(l,coefficients(basepoly,monomials(var,d)))
        end
        l=transpose(reduce(vcat,transpose.(l)))
        push!(c,l)
    end
    c=transpose(reduce(vcat,transpose.(c)))
    
    #Create Linear Program
    m = size(b,1) #Number of Monomials in Polynom to Match
    LP = Model(HiGHS.Optimizer) # Initialize Model
    n=size(c,2) # Number of Variables
    @variable(LP, u[1:n]>=0)
    for i in 1:m
        @constraint(LP, sum(dot(u,c[i,:])) == b[i])
    end
    print(LP)
    return (LP,u,b,c,nVars)
end

getLP (generic function with 1 method)